# Лабораторная работа 4 (RandomForest)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from utils import regression_cross_validate, display_metrics_table, classification_cross_validate, display_metrics_classification_table
import warnings
warnings.filterwarnings("ignore")

### Regression

#### 1. Обработка данных

In [2]:
df = pd.read_csv('data/laptop_prices.csv')
df.head()

,Company,Product,TypeName,Inches,Ram,OS,Weight,Price_euros,Screen,ScreenW,...,RetinaDisplay,CPU_company,CPU_freq,CPU_model,PrimaryStorage,SecondaryStorage,PrimaryStorageType,SecondaryStorageType,GPU_company,GPU_model
0,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1339.69,Standard,2560,...,Yes,Intel,2.3,Core i5,128,0,SSD,No,Intel,Iris Plus Graphics 640
1,Apple,Macbook Air,Ultrabook,13.3,8,macOS,1.34,898.94,Standard,1440,...,No,Intel,1.8,Core i5,128,0,Flash Storage,No,Intel,HD Graphics 6000
2,HP,250 G6,Notebook,15.6,8,No OS,1.86,575.00,Full HD,1920,...,No,Intel,2.5,Core i5 7200U,256,0,SSD,No,Intel,HD Graphics 620
3,Apple,MacBook Pro,Ultrabook,15.4,16,macOS,1.83,2537.45,Standard,2880,...,Yes,Intel,2.7,Core i7,512,0,SSD,No,AMD,Radeon Pro 455
4,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1803.60,Standard,2560,...,Yes,Intel,3.1,Core i5,256,0,SSD,No,Intel,Iris Plus Graphics 650


In [3]:
features, target = df.drop(columns=['Price_euros']), df['Price_euros']

numerical_features = features.select_dtypes(include=[np.number]).columns  # Численные признаки
categorical_features = features.select_dtypes(exclude=[np.number]).columns  # Категориальные признаки

train_features, test_features, train_target, test_target = train_test_split(features, target, random_state=42)

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=-1)

train_features[categorical_features] = le.fit_transform(train_features[categorical_features])
test_features[categorical_features] = le.transform(test_features[categorical_features])

imputer = SimpleImputer(strategy='most_frequent') 
train_features = pd.DataFrame(imputer.fit_transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(imputer.transform(test_features), columns=test_features.columns)

#### 2. Построение бейзлайна 

Для оценки модели будем использовать метод кросс валидации, который позволяет более качественно оценить полученные метрики.

In [4]:
metrics = regression_cross_validate(RandomForestRegressor, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, random_state=42, n_estimators=100, max_depth=3)
display_metrics_table(*metrics)

rf = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=3)
rf.fit(train_features, train_target)

predicted_target = rf.predict(test_features)

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      |    282.472    |    20.1201    |
| MSE      | 170966        | 21337.5       |
| R2       |      0.650488 |     0.0421918 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 153734.93
Средняя абсолютная ошибка (MAE): 294.86
Коэффициент детерминации (R^2): 0.68


Как можем увидеть значение метрики $R^2$ около 0.68, что означает что около 68% дисперсии данных объясняется моделью.

#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Увеличить глубину и число деревьев

In [5]:
onehot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

encoded_train_data = onehot.fit_transform(train_features[categorical_features])
encoded_test_data = onehot.transform(test_features[categorical_features])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(categorical_features))
updated_train_features = train_features.drop(columns=categorical_features).reset_index(drop=True)
updated_train_features = pd.concat([updated_train_features, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(categorical_features))
updated_test_features = test_features.drop(columns=categorical_features).reset_index(drop=True)
updated_test_features = pd.concat([updated_test_features, encoded_df], axis=1)

# scaler = StandardScaler()
# updated_train_features[numerical_features] = scaler.fit_transform(train_features[numerical_features])
# updated_test_features[numerical_features] = scaler.transform(test_features[numerical_features])

In [6]:
metrics = regression_cross_validate(DecisionTreeRegressor, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, max_depth=10, random_state=42)
display_metrics_table(*metrics)

dt = DecisionTreeRegressor(max_depth=10, random_state=42)
dt.fit(updated_train_features, train_target)

predicted_target = dt.predict(updated_test_features)

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |      Std Dev |
|:---------|--------------:|-------------:|
| MAE      |    230.343    |    16.5754   |
| MSE      | 145682        | 28143.9      |
| R2       |      0.702678 |     0.054038 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 118536.23
Средняя абсолютная ошибка (MAE): 228.46
Коэффициент детерминации (R^2): 0.76


Можно увидеть, что в целом значения метрик улучшились. Результаты на тестовой выборке также показывают приросты.

#### 4. Реализация своего класса

In [7]:
class MyRandomForestRegressor:
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators  # Количество деревьев
        self.max_depth = max_depth  # Глубина дерева
    def fit(self, features, target):
        self.trees = []
        for _ in range(self.n_estimators):
            bootstrap_indices = np.random.choice(len(features), size=len(features), replace=True)
            X_bootstrap = features[bootstrap_indices]
            y_bootstrap = target[bootstrap_indices]
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X_bootstrap, y_bootstrap)
            self.trees.append(tree)

    def predict(self, features):
        predictions = np.zeros((len(features), self.n_estimators))
        for i, tree in enumerate(self.trees):
            predictions[:, i] = tree.predict(features)
        return predictions.mean(axis=1)

In [8]:
metrics = regression_cross_validate(MyRandomForestRegressor, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_estimators=100, max_depth=3)
display_metrics_table(*metrics)

rf = MyRandomForestRegressor(n_estimators=100, max_depth=3)
rf.fit(train_features.to_numpy(), train_target.to_numpy())

predicted_target = rf.predict(test_features.to_numpy())

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |          Mean |       Std Dev |
|:---------|--------------:|--------------:|
| MAE      |    281.003    |    18.8823    |
| MSE      | 167922        | 22305.2       |
| R2       |      0.656711 |     0.0443118 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 153643.00
Средняя абсолютная ошибка (MAE): 294.54
Коэффициент детерминации (R^2): 0.68


In [9]:
metrics = regression_cross_validate(MyRandomForestRegressor, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_estimators=300, max_depth=7)
display_metrics_table(*metrics)

rf = MyRandomForestRegressor(n_estimators=300, max_depth=7)
rf.fit(train_features.to_numpy(), train_target.to_numpy())

predicted_target = rf.predict(test_features.to_numpy())

# Метрики
mse = mean_squared_error(test_target, predicted_target)
mae = mean_absolute_error(test_target, predicted_target)
r2 = r2_score(test_target, predicted_target)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |         Mean |       Std Dev |
|:---------|-------------:|--------------:|
| MAE      |    203.802   |     7.10218   |
| MSE      | 109069       | 10029.9       |
| R2       |      0.77617 |     0.0282656 |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 125749.74
Средняя абсолютная ошибка (MAE): 233.22
Коэффициент детерминации (R^2): 0.74


Результаты показывают что собственная имплементация модели в среднем работает на том же уровне качества, что и модель из `sklearn`

### Classification

#### 1. Обработка данных

In [10]:
df = pd.read_csv('data/AIDS_Classification.csv')
df.head()

,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820,infected
0,948,2,48,89.8128,0,0,0,100,0,0,...,0,1,0,1,0,422,477,566,324,0
1,1002,3,61,49.4424,0,0,0,90,0,1,...,1,3,0,1,0,162,218,392,564,1
2,961,3,45,88.4520,0,1,1,90,0,1,...,1,3,0,1,1,326,274,2063,1893,0
3,1166,3,47,85.2768,0,1,0,100,0,1,...,1,3,0,1,0,287,394,1590,966,0
4,1090,0,43,66.6792,0,1,0,100,0,1,...,1,3,0,0,0,504,353,870,782,0


In [11]:
features, target = df.drop(columns=['infected']), df['infected']

train_features, test_features, train_target, test_target = train_test_split(features, target, random_state=42)

num_features = features.select_dtypes(include=[np.number]).columns  # Численные признаки
categorical_features = features.select_dtypes(exclude=[np.number]).columns  # Категориальные признаки

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=99)

train_features[categorical_features] = le.fit_transform(train_features[categorical_features])
test_features[categorical_features] = le.transform(test_features[categorical_features])

imputer = SimpleImputer(strategy='most_frequent') 
train_features = pd.DataFrame(imputer.fit_transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(imputer.transform(test_features), columns=test_features.columns)

#### 2. Построение бейзлайна 

In [12]:
metrics = classification_cross_validate(RandomForestClassifier, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, random_state=42, n_estimators=100, max_depth=3)
display_metrics_classification_table(*metrics)

rf = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=3)
rf.fit(train_features, train_target)

predicted_target = rf.predict(test_features)

accuracy = accuracy_score(test_target, predicted_target)
precision = precision_score(test_target, predicted_target, average='weighted')
recall = recall_score(test_target, predicted_target, average='weighted')
f1 = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.822329 | 0.0248596 |
| Precision | 0.839277 | 0.0158632 |
| Recall    | 0.822329 | 0.0248596 |
| F1-score  | 0.786084 | 0.033736  |

=== Результаты на Тесте ===
1. Accuracy: 81.50%
2. Precision: 83.44%
3. Recall: 81.50%
4. F1-score: 77.30%


#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Увеличить глубину и число деревьев

In [16]:
features, target = df.drop(columns=['infected']), df['infected']

train_features, test_features, train_target, test_target = train_test_split(features, target, random_state=42)

num_features = features.select_dtypes(include=[np.number]).columns  # Численные признаки
categorical_features = features.select_dtypes(exclude=[np.number]).columns  # Категориальные признаки

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=99)

train_features[categorical_features] = le.fit_transform(train_features[categorical_features])
test_features[categorical_features] = le.transform(test_features[categorical_features])

imputer = SimpleImputer(strategy='most_frequent') 
updated_train_features = pd.DataFrame(imputer.fit_transform(train_features), columns=train_features.columns)
updated_test_features = pd.DataFrame(imputer.transform(test_features), columns=test_features.columns)

In [17]:
metrics = classification_cross_validate(RandomForestClassifier, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, random_state=42, n_estimators=300, max_depth=10)
display_metrics_classification_table(*metrics)

rf = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42)
rf.fit(updated_train_features.to_numpy(), train_target.to_numpy())

predicted_target = rf.predict(updated_test_features.to_numpy())

accuracy = accuracy_score(test_target, predicted_target)
precision = precision_score(test_target, predicted_target, average='weighted')
recall = recall_score(test_target, predicted_target, average='weighted')
f1 = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")


| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.894026 | 0.0145424 |
| Precision | 0.891956 | 0.0154455 |
| Recall    | 0.894026 | 0.0145424 |
| F1-score  | 0.891469 | 0.0151135 |

=== Результаты на Тесте ===
1. Accuracy: 89.16%
2. Precision: 89.09%
3. Recall: 89.16%
4. F1-score: 88.52%


Можно заметить, что в среднем показатели метрик улучшились по сравнению с предыдущими результатами. Анализ результатов на тестовой выборке также демонстрирует существенные улучшения, что подтверждает эффективность проведённых модификаций.

#### 4. Реализация своего класса

In [18]:
class MyRandomForestClassifier:
    def __init__(self, n_estimators=100, max_features='sqrt', max_depth=None, min_samples_split=2):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def bootstrap_sample(self, features, target):
        n_samples = features.shape[0]
        indices = np.random.choice(n_samples, size=n_samples, replace=True)
        return features[indices], target[indices]

    def fit(self, features, target):
        self.trees = []
        self.feature_indices = []

        for _ in range(self.n_estimators):
            # Создаем бутстреп-выборку
            X_sample, y_sample = self.bootstrap_sample(features, target)

            # Создаем и обучаем дерево
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, features):
        tree_preds = np.array([tree.predict(features) for tree in self.trees])
        # Возвращаем большинство голосов
        predictions = np.apply_along_axis(lambda x: np.bincount(x, minlength=len(np.unique(x))).argmax(), axis=0, arr=tree_preds)
        return predictions

In [19]:
metrics = classification_cross_validate(MyRandomForestClassifier, train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_estimators=100, max_depth=3)
display_metrics_classification_table(*metrics)

rf = MyRandomForestClassifier(n_estimators=100, max_depth=3)
rf.fit(train_features.to_numpy(), train_target.to_numpy())

predicted_target = rf.predict(test_features.to_numpy())

accuracy = accuracy_score(test_target, predicted_target)
precision = precision_score(test_target, predicted_target, average='weighted')
recall = recall_score(test_target, predicted_target, average='weighted')
f1 = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.89028  | 0.0151447 |
| Precision | 0.888657 | 0.0177813 |
| Recall    | 0.89028  | 0.0151447 |
| F1-score  | 0.885198 | 0.0154108 |

=== Результаты на Тесте ===
1. Accuracy: 88.04%
2. Precision: 88.39%
3. Recall: 88.04%
4. F1-score: 86.96%


In [20]:
metrics = classification_cross_validate(MyRandomForestClassifier, updated_train_features.to_numpy(), train_target.to_numpy(), n_folds=5, n_estimators=300, max_depth=10)
display_metrics_classification_table(*metrics)

rf = MyRandomForestClassifier(n_estimators=300, max_depth=10)
rf.fit(updated_train_features.to_numpy(), train_target.to_numpy())

predicted_target = rf.predict(updated_test_features.to_numpy())

accuracy = accuracy_score(test_target, predicted_target)
precision = precision_score(test_target, predicted_target, average='weighted')
recall = recall_score(test_target, predicted_target, average='weighted')
f1 = f1_score(test_target, predicted_target, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")


| Metric    |     Mean |   Std Dev |
|:----------|---------:|----------:|
| Accuracy  | 0.895273 | 0.0223009 |
| Precision | 0.893211 | 0.0237673 |
| Recall    | 0.895273 | 0.0223009 |
| F1-score  | 0.893376 | 0.0233035 |

=== Результаты на Тесте ===
1. Accuracy: 88.41%
2. Precision: 88.23%
3. Recall: 88.41%
4. F1-score: 87.73%


Результаты показывают, что собственная реализация модели в среднем такой же по качеству что и модели из `sklearn`, однако демонстрирует приросты при использовании улучшенного бейзлайна.

### Заключение

Внесённые изменения, включая нормализацию данных и увеличение глубины и числа деревьев в алгоритме, улучшают точность модели. Проведённые эксперименты демонстрируют, что как собственная реализация, так и применение моделей из sklearn, дают схожие результаты.

| Модель                    |      MSE  |        MAE |      $R^2$ |
|:--------------------------|----------:|-----------:|-----------:|
| Sklearn (до улучшения)    | 153734.93  | 294.86   |  0.68      |
| Sklearn (после улучшения) | 118536.23  | 228.46   |  0.76      |
| Собственная имплементация (до улучшения)   | 155908.88   | 297.55    |  0.68      |
| Собственная имплементация (после улучшения)| 121329.04  | 231.17    |  0.75      |

| Модель                    |  Accuracy |  Precision |     Recall |    F1-score |
|:--------------------------|----------:|-----------:|-----------:|-----------:|
| Sklearn (до улучшения)    |   81.50%  |   83.44%   |  81.50%    |  77.30%    |
| Sklearn (после улучшения) |   89.16%  |   89.09%   |  89.16%    |  88.52%    |
| Собственная имплементация (до улучшения)   |   88.04%  |   88.39%   |  88.04%    |  86.96%    |
| Собственная имплементация (после улучшения)|   88.41%  |   88.23%  |  88.41%    |  87.73%    |